## DKRZ NCL notebook example
<table align="left">
<tr><td>Title:</td><td>Regridding</td></tr>
<tr><td>Description</td><td>Regrid data from curvilinear grid to coarse 1x1 degree rectilinear grid</td></tr>
<tr><td>23.07.18</td><td>kmf</td></tr>
</table>

Load the ESMF library. **Note: since v6.5.0 automatically loaded.**

In [15]:
load "$NCARG_ROOT/lib/ncarg/nclscripts/esmf/ESMF_regridding.ncl"

Open file and read the variable tos. The data set is included in the NCL package. The data set tos_ocean_bipolar_grid.nc contains data on a curvilinear grid (latitude and longitude coordinate variables are 2-dimensional).

In [16]:
f   =  addfile("$NCARG_ROOT/lib/ncarg/data/nug/tos_ocean_bipolar_grid.nc","r")
tos =  f->tos(0,:,:)

When the lat and lon coordinate variables are 2-dimensional adding the attributes lat2d and lon2d to the variable tells NCL what kind of source grid we have.

In [17]:
tos@lat2d =  f->lat
tos@lon2d =  f->lon

Print the metadata summary of variable tos. **Note: The NCL Kernel is not displaying the output of printVarSummary. You can download the script and run it in a terminal and it will show the printVarSummary results).**

In [18]:
printVarSummary(tos)

Set ESMF resources.

In [19]:
Opt                 =  True
Opt@InterpMethod    = "bilinear"
Opt@ForceOverwrite  =  True
Opt@SrcMask2D       =  where(.not. ismissing(tos),1,0)
Opt@DstGridType     = "1x1"                     ;-- Destination grid
Opt@DstLLCorner     =  (/-89.75d,   0.00d /)
Opt@DstURCorner     =  (/ 89.75d, 359.75d /) 

Regrid variable tos to 1x1 degrees rectilinear grid.

In [20]:
tos_regrid = ESMF_regrid(tos,Opt)

Print the metadata summary of the regridded variable tos_regrid. **Note: The NCL Kernel is not displaying the output of printVarSummary. You can download the script and run it in a terminal and it will show the printVarSummary results).**

In [21]:
printVarSummary(tos_regrid)

Send graphic output to PNG file.

In [22]:
wks = gsn_open_wks("png","plot_regridding")

Set plot resources.

In [23]:
res                =  True
res@gsnDraw        =  False                   ;-- don't draw the plot, yet
res@gsnFrame       =  False                   ;-- don't advance the frame

res@cnFillOn       =  True                    ;-- contour fill 
res@cnFillPalette  = "cmp_b2r"
res@cnLinesOn      =  False                   ;-- turn off contour lines
res@cnLineLabelsOn =  False                   ;-- turn off contour labels
res@cnFillMode     = "RasterFill"             ;-- turn raster fill on

res@lbLabelBarOn   =  False                   ;-- don't draw a labelbar; gsn_panel will do it later

Set plot title and create the contour plot of variable tos in memory.

In [24]:
res@tiMainString  = "Original data"

plot_source = gsn_csm_contour_map(wks,tos,res)

Set plot title and create the contour plot of variable tos_regrid in memory.

In [25]:
res@tiMainString  = "Regridded data"
                     
plot_regrid = gsn_csm_contour_map(wks,tos_regrid,res)

Set panel resources.

In [26]:
pres                     =  True
pres@gsnMaximize         =  True                    ;-- maximize graphic output
pres@gsnPanelLabelBar    =  True                    ;-- draw common labelbar
pres@lbLabelFontHeightF  =  0.01

Create the panel plot.

In [27]:
gsn_panel(wks,(/plot_source,plot_regrid/),(/2,1/),pres)

![regridding](plot_regridding.png)